In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [3]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
sync_filt = Filter.create_filter(address=None, event_types=[Pair.events.Sync])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=sync_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])
    pair: PairDetails = FetchPairDetails().apply(web3, evt['address'])

    amt0 = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    amt1 = Conversion().convert_int256_bytes_to_int(arguments[1])  
    amt0_human = amt0/(10**pair.token0.decimals)
    amt1_human = amt1/(10**pair.token1.decimals)    

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt['event']
    event['details']['token0'] = pair.token0.address
    event['details']['token1'] = pair.token1.address
    event['details']['token0_symbol'] = pair.token0.symbol
    event['details']['token1_symbol'] = pair.token1.symbol 
    event['details']['token0_decimal'] = pair.token0.decimals
    event['details']['token1_decimal'] = pair.token1.decimals        
    event['details']['amount0'] = amt0
    event['details']['amount1'] = amt1
    event['details']['price'] = amt1_human/amt0_human
    
    events[k] = event
    if key not in processed_events:
        print(f"Sync at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Sync at block:60,879,582 tx:0x38c5ab3ff4fa157990d90d87e72d92b4f25fcac8a05698ea27b1e1893b674a72
Sync at block:60,879,586 tx:0xe8ceda205a403ab089a11007377bc9a4559265b842298024cc91c295cc153e8c
Sync at block:60,879,589 tx:0xdc868f939e847d49c46e672f095b01fe913e316f2ad5243bb4c32a7a723fb8a3
Sync at block:60,879,589 tx:0xdc868f939e847d49c46e672f095b01fe913e316f2ad5243bb4c32a7a723fb8a3
Sync at block:60,879,589 tx:0xdc868f939e847d49c46e672f095b01fe913e316f2ad5243bb4c32a7a723fb8a3
Sync at block:60,879,589 tx:0xc35436075beaf8d962c1ed056857a6e00c5e765510ac3e55714a1b0174d298d9
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,sync,sushi,0xff2bbcb399ad50bbd06debadd47d290933ae1038,0x38c5ab3ff4fa157990d90d87e72d92b4f25fcac8a056...,60879582,1724284524,{'web3_type': <class 'web3._utils.datatypes.Sy...
1,polygon,uniswapv2pair,sync,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0xe8ceda205a403ab089a11007377bc9a4559265b84229...,60879586,1724284534,{'web3_type': <class 'web3._utils.datatypes.Sy...
2,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xdc868f939e847d49c46e672f095b01fe913e316f2ad5...,60879589,1724284540,{'web3_type': <class 'web3._utils.datatypes.Sy...
3,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xdc868f939e847d49c46e672f095b01fe913e316f2ad5...,60879589,1724284540,{'web3_type': <class 'web3._utils.datatypes.Sy...
4,polygon,uniswapv2pair,sync,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xdc868f939e847d49c46e672f095b01fe913e316f2ad5...,60879589,1724284540,{'web3_type': <class 'web3._utils.datatypes.Sy...
5,polygon,uniswapv2pair,sync,sushi,0xa5c7cc743b6d0bc986fc8cd4642f593dd3c69a5c,0xc35436075beaf8d962c1ed056857a6e00c5e765510ac...,60879589,1724284540,{'web3_type': <class 'web3._utils.datatypes.Sy...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0xff2bbcb399ad50bbd06debadd47d290933ae1038',
  'tx_hash': '0x38c5ab3ff4fa157990d90d87e72d92b4f25fcac8a05698ea27b1e1893b674a72',
  'blk_num': 60879582,
  'timestamp': 1724284524,
  'details': {'web3_type': web3._utils.datatypes.Sync,
   'token0': '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
   'token1': '0xC3Ec80343D2bae2F8E680FDADDe7C17E71E114ea',
   'token0_symbol': 'WETH',
   'token1_symbol': 'OM',
   'token0_decimal': 18,
   'token1_decimal': 18,
   'amount0': 79710744325904784655,
   'amount1': 233079035947315535981558,
   'price': 2924.0604628448864}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'sync',
  'platform': 'sushi',
  'address': '0x604229c960e5cacf2aaeac8be68ac07ba9df81c3',
  'tx_hash': '0xe8ceda205a403ab089a11007377bc9a4559265b842298024cc91c295cc153e8c',
  'blk_num': 60879586,
  'timestamp': 1724284534,
  'details': {'web3_type': web3._ut